## Import functions and define universal values

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.insert(0,'./code')
import run_higher_sph_harm_prefit
import plot_utils_deglist

#May need to add things

lcName='W18_NIRISS_spec_lambin_25_prefit_test_16_v3'
planetparams={}
planetparams['t0']=2459802.4078798565 #units of days
planetparams['per']=0.941452382 #units of days
planetparams['a_abs']=0.0218 #units of AU
planetparams['inc']=84.35320 #units of degrees
planetparams['ecc']=0.0 #unitless
planetparams['w']=269. #units of degrees
planetparams['rprs']=0.09783 #unitless ratio
planetparams['ars']=3.48023 #unitless ratio
planetparams['t_sec']=2459802.8786060475 #units of days


## Load in data and set up data dictionary

In [ ]:
waveind=16 #for fitting a single wavelength
specfile=np.load('./real_spec/bin25/ind_bins_'+str(waveind)+'.npz')
#specfile=np.load('spec_lambin_25.npz')
time=specfile['arr_0']
#waves=specfile['arr_1']
waves=np.array([specfile['arr_1']])
fluxes=specfile['arr_2']+10**6.
errs=specfile['arr_3']
extent=np.zeros(2)
extent[0]=(np.min(time)-planetparams['t_sec'])/planetparams['per']*2.*np.pi-np.pi/2. #minimum observed point, in radians
extent[1]=(np.max(time)-planetparams['t_sec'])/planetparams['per']*2.*np.pi+np.pi/2. #maximum observed point, in radians
print(extent*180./np.pi)

plt.figure()
plt.errorbar(time,fluxes,yerr=errs,color='k',marker='.',linestyle='none')
plt.show()

datadict={"time (days)":time, "wavelength (um)":waves, "flux (ppm)": fluxes, "flux err (ppm)": errs}


## Perform eigencurve fitting at each wavelength

In [ ]:

#Test fitting with spherical harmonics to order=2,3,4
#Afew>=20 will fit for the correct number of coefficients
saveDir = "./data/sph_harmonic_coefficients_full_samples/" + lcName
for oneOrd in np.arange(3,7):
    for afew in np.arange(2,7):
        run_higher_sph_harm_prefit.run_lc_fit(datadict,planetparams,norder=oneOrd,lcName=lcName,saveDir=saveDir,\
                                              afew=int(afew),burnin=3000,nsteps=30000,plot=True,strict=False,nonegs=True)


## Plot 2D maps

In [ ]:
dataDir=saveDir+'/'
for i in np.arange(20):
    waves, lats, lons, mapLowMedHigh = plot_utils.get_map_and_plot(planetparams,dataDir,afew=20,waveInd=0,degree=3,isspider=False,saveName='W18_spec')
#FINDME: using a different waveInd will show maps at a different wavelength
#using a different degree will show a fit with a different number of spherical harmonics
#isspider=False uses Arthur's implementation of converting from lightcurves to maps (faster than the SPIDERMAN method and identical maps are produced)
#The middle map (labeled 50th percentile map) shows the single best-fit map with the lowest chi-squared

## Converting Fp/Fs map into planet temperature map

In [ ]:
Ts=6400.
# c=2.998*10**8.
# h=6.626*10**-34.
# kb=1.381*10**-23.
# lam=1.7*10**-6.
rprs=planetparams['rprs']
medianmap=mapLowMedHigh[1][0]
medianmap[medianmap<0.]=0. #FINDME: remove this if you want the real fluxes instead of removing fluxes below zero
mapinTemp=plot_utils.plot_map_in_temp(Ts,lam,dlam,rprs,lats,lons,medianmap,extent,waveInd=0,degree=3,plot=True,saveName='W18_spec')


## Calculate brightest point in longitude and latitude

In [ ]:
hotlats,hotlons = plot_utils.find_hotspot(planetparams,dataDir,afew=20,waveInd=0,degree=3,saveName='W18_spec',isspider=False)
